In [ ]:
"""
this notebook converts all alignedFaces into standardized
fixed flattened sampling vertices face data
save them in a csv file
"""
import trimesh
import yaml
import pathlib
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas
import numpy as np

In [113]:
NUM_SAMPLES = 2000
def preprocess(face_path:str, num_samples=NUM_SAMPLES) -> np.ndarray:
    """
    return standardized fixed sampling vertices of given face in tensor form
    keep num_samples small to speed up training
    num_samples is ideally less than minimum number of vertices of all given face
    so that size of tensor will be consistent
    """
    face = trimesh.load_mesh(face_path)
    # try random sampling to reduce points?
    face_vertices = face.sample(num_samples)
    vertices_tensor = np.asarray(face_vertices)
    scaler = StandardScaler()
    # standardization
    scaled_vertices = scaler.fit_transform(vertices_tensor)
    # flatten it?
    flattened_scaled_vertices = np.reshape(scaled_vertices, newshape=(3*num_samples))
    return flattened_scaled_vertices



In [114]:

def draw_sampled_face(scaled_vertices:np.ndarray):
    """
    visualize 3d point cloud
    """
    scaled_vertices = np.reshape(scaled_vertices, newshape=(NUM_SAMPLES, 3))
    x = scaled_vertices[:,0]
    y = scaled_vertices[:,1]
    z = scaled_vertices[:,2]
    ax = plt.axes(projection="3d")
    ax.scatter(x,y,z)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    plt.show()

In [115]:
# process all faces
path = pathlib.Path("alignedFaces/")
pathlist = list(path.glob("*.obj"))
data = dict()
for face_path in pathlist:
    face_tensor = preprocess(face_path, num_samples=NUM_SAMPLES)
    data.update({str(face_path.stem): face_tensor})


In [119]:

df  = pandas.DataFrame()
df  = df.from_dict(data, orient="index")
df.to_csv("preprocessed_data.csv")